In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
import json
from tqdm import tqdm
import os

In [2]:
import nomad.io.base as loader
import nomad.city_gen as cg
import nomad.traj_gen as tg
from nomad.traj_gen import Agent, Population, garden_city_to_mercator
from nomad.city_gen import City

## Load city and configure destination diaries

In [6]:
# Load the city
city_file = '../../garden-city.gpkg'
city = cg.City.from_geopackage(city_file)
start = '2024-06-01 00:00-04:00'

#option 1: symmetric
start_time = pd.date_range(start=start, periods=4, freq='60min')
unix_timestamp = [int(t.timestamp()) for t in start_time]
duration = [60]*4  # in minutes
location = ['h-x13-y11'] * 1 + ['h-x13-y9'] * 1 + ['w-x18-y10'] * 1 + ['w-x18-y8'] * 1

destinations = pd.DataFrame(
    {
        "datetime":start_time,
         "timestamp":unix_timestamp,
         "duration":duration,
         "location":location
    }
)
destinations.to_csv("exp_1_destinations_balanced.csv", index=False)

#option 2: we break symmetries to produce more density heterogeneity
start_time = pd.date_range(start=start, periods=8, freq='30min')
unix_timestamp = [int(t.timestamp()) for t in start_time]
duration = [30]*8 
location = ['h-x13-y11'] * 2 + ['h-x13-y9'] * 1 + ['w-x18-y10'] * 2 + ['w-x18-y8'] * 3

destinations = pd.DataFrame(
    {
        "datetime":start_time,
         "timestamp":unix_timestamp,
         "duration":duration,
         "location":location
    }
)
destinations.to_csv("exp_1_destinations_unbalanced.csv", index=False)

## Config files for simulations
This could be in a json or yaml and should be passable to a Population object.

In [11]:
# option 1 (reduced for quick demo run)
N_reps = 2
sparsity_samples = 3
config = dict(
    dt = 0.5,
    N = N_reps*sparsity_samples,
    name_count=2,
    name_seed=2025,
    city_file='../../garden-city.gpkg',
    destination_diary_file='exp_1_destinations_balanced.csv',
    output_files = dict(
        sparse_path='./sparse_traj_1',
        diaries_path='./diaries_1',
        homes_path='./homes_1'
    ),
    agent_params = dict(
        agent_homes='h-x13-y11',
        agent_workplaces='w-x18-y8',
        seed_trajectory=list(range(N_reps*sparsity_samples)),
        seed_sparsity= list(range(N_reps*sparsity_samples)),
        beta_ping= np.repeat(np.linspace(1, 20, sparsity_samples), N_reps).tolist(),
        beta_durations=None,
        beta_start=None,
        ha=11.5/15
    )
)
with open('config_low_ha.json', 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=4)

# option 2 (reduced for quick demo run)
N_reps = 2
sparsity_samples = 3
config_2 = dict(
    dt = 0.5,
    N = N_reps*sparsity_samples,
    name_count=2,
    name_seed=2025,
    city_file='../../garden-city.gpkg',
    destination_diary_file='exp_1_destinations_unbalanced.csv',
    output_files = dict(
        sparse_path='./sparse_traj_2',
        diaries_path='./diaries_2',
        homes_path='./homes_2'
    ),
    agent_params = dict(
        agent_homes='h-x13-y11',
        agent_workplaces='w-x18-y8',
        seed_trajectory=list(range(N_reps*sparsity_samples)),
        seed_sparsity= list(range(N_reps*sparsity_samples)),
        beta_ping= np.repeat(np.linspace(1, 20, sparsity_samples), N_reps).tolist(),
        beta_durations=None,
        beta_start=None,
        ha=15/15
    )
)
with open('config_high_ha.json', 'w', encoding='utf-8') as f:
    json.dump(config_2, f, ensure_ascii=False, indent=4)

## Generate trajectories

In [14]:
city.buildings_gdf

,id,type,door_x,door_y,door_cell_x,door_cell_y,door_point,size,geometry
0,p-x13-y11,park,13.0,11.5,13,11,POINT (13 11.5),16,"POLYGON ((13 9, 13 13, 9 13, 9 9, 13 9))"
1,h-x8-y8,home,8.0,8.5,8,8,POINT (8 8.5),2,"POLYGON ((7 7, 8 7, 8 9, 7 9, 7 7))"
2,h-x9-y8,home,9.5,8.0,9,8,POINT (9.5 8),2,"POLYGON ((8 7, 10 7, 10 8, 8 8, 8 7))"
3,h-x10-y8,home,10.5,8.0,10,8,POINT (10.5 8),1,"POLYGON ((11 7, 11 8, 10 8, 10 7, 11 7))"
4,h-x11-y8,home,11.5,8.0,11,8,POINT (11.5 8),1,"POLYGON ((12 7, 12 8, 11 8, 11 7, 12 7))"
...,...,...,...,...,...,...,...,...,...
101,r-x3-y7,retail,3.0,7.5,3,7,POINT (3 7.5),2,"POLYGON ((3 7, 3 8, 1 8, 1 7, 3 7))"
102,r-x0-y5,retail,1.0,5.5,0,5,POINT (1 5.5),3,"POLYGON ((2 4, 2 7, 1 7, 1 4, 2 4))"
103,r-x3-y6,retail,3.0,6.5,3,6,POINT (3 6.5),1,"POLYGON ((3 6, 3 7, 2 7, 2 6, 3 6))"
104,r-x3-y5,retail,3.0,5.5,3,5,POINT (3 5.5),1,"POLYGON ((3 5, 3 6, 2 6, 2 5, 3 5))"


In [12]:
# Parameters according to the config file
with open('config_high_ha.json', 'r', encoding='utf-8') as f:
    config = json.load(f)
    
# Load city and destination diary from config
city = City.from_geopackage(config["city_file"])
poi_data = city.get_building_coordinates() # and type and size

destinations = pd.read_csv(config["destination_diary_file"], parse_dates=["datetime"])

population = Population(city)
population.generate_agents(
    N=config["N"], 
    seed=config["name_seed"], 
    name_count=config["name_count"],
    agent_homes=config["agent_params"]["agent_homes"],
    agent_workplaces=config["agent_params"]["agent_workplaces"]
)

for i, agent in enumerate(tqdm(population.roster.values(), desc="Generating trajectories")):
    agent.generate_trajectory(
        destination_diary=destinations,
        dt=config["dt"],
        seed=config["agent_params"]["seed_trajectory"][i],
        step_seed=config["agent_params"]["seed_trajectory"][i])
    
    agent.sample_trajectory(
        beta_ping=config["agent_params"]["beta_ping"][i],
        seed=config["agent_params"]["seed_sparsity"][i],
        ha=config["agent_params"]["ha"],
        replace_sparse_traj=True)

# Reproject all trajectories to Web Mercator at population level
print("Reprojecting trajectories to Web Mercator...")
population.reproject_to_mercator(sparse_traj=True, full_traj=False, diaries=True, poi_data=poi_data)

Generating trajectories: 100%|███████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.94it/s]

Reprojecting trajectories to Web Mercator...


In [13]:
# Save output files using save_pop method
print("Saving output files...")
population.save_pop(
    sparse_path=config["output_files"]["sparse_path"],
    diaries_path=config["output_files"]["diaries_path"],
    homes_path=config["output_files"]["homes_path"],
    beta_ping=config["agent_params"]["beta_ping"],
    ha=config["agent_params"]["ha"]
)
print("All output files saved successfully!")

Saving output files...
All output files saved successfully!
